# Ways To Build And Manage A Computation Graph in Python

## 1. Computation Graph using built-in Python Properties

In [1]:
from math import pi
from itertools import chain

class Cylinder:

    _dependencies = {
        "length": ["volume"],
        "radius": ["volume"],
        "volume": ["mass"],
        "density": ["mass"]
    }
    _dependent_vars = set(chain(*list(_dependencies.values())))

    def __init__(self, radius, length, density):
        self._radius = radius
        self._length = length
        self._density = density
        self._volume = None
        self._mass = None

    def _reset_dependent_vars(self, name):
        for var in self._dependencies[name]:
            super().__setattr__(f"_{var}", None)
            if var in self._dependencies:
                self._reset_dependent_vars(var)

    def __setattr__(self, name, value):
        if name in self._dependent_vars:
            raise AttributeError("Cannot set this value.")
        if name in self._dependencies:
            self._reset_dependent_vars(name)
            name = f"_{name}"
        super().__setattr__(name, value)

    @property
    def volume(self):
        """Calculates cylinder's volume."""
        if self._volume is None:
            self._volume = self.length*pi*self.radius**2
            print("Volume calculated")
        return self._volume

    @property
    def mass(self):
        """Calculates cylinder's mass."""
        if self._mass is None:
            self._mass = self.volume*self.density
            print("Mass calculated")
        return self._mass

    @property
    def length(self):
        return self._length

    @property
    def radius(self):
        return self._radius

    @property
    def density(self):
        return self._density

In [2]:
c = Cylinder(0.25, 1.0, 450)

In [3]:
c.radius

0.25

In [4]:
c.length

1.0

In [5]:
c.density

450

In [6]:
c.volume

Volume calculated


0.19634954084936207

In [7]:
c.mass

Mass calculated


88.35729338221293

In [8]:
c.length = c.length*2  # This should change things!

In [9]:
c.mass

Volume calculated
Mass calculated


176.71458676442586

In [10]:
c.volume

0.39269908169872414

In [11]:
try:
    c.volume = 0
except AttributeError as err:
    print(err)

Cannot set this value.


## 2. Trying out pythonflow package to build computation graph

In [12]:
import pythonflow as pf
import math

with pf.Graph() as graph:
    pi = pf.constant(math.pi)
    length = pf.constant(1.0)
    radius = pf.constant(0.25)
    density = pf.constant(450)
    volume = length*pi*radius**2
    mass = volume*density

In [13]:
graph(volume)

0.19634954084936207

In [14]:
graph(mass)

88.35729338221293

In [15]:
graph(volume, {length: graph(length)*2})

0.39269908169872414

In [16]:
graph(mass, {length: graph(length)*2})

176.71458676442586

In [17]:
graph(mass, {length: graph(length)*2})

176.71458676442586

In [18]:
# Problem is, I think it is still recalculating all dependencies each time...


## 3. Using Pythonflow with Class Properties

In [19]:
from math import pi

class Cylinder:

    _independent = {"length", "radius", "density"}
    _dependent = {"volume", "mass"}

    def __init__(self, radius, length, density):
        self._length = length
        self._radius = radius
        self._density = density
        self._volume = None
        self._mass = None
        with pf.Graph() as graph:
            self._pf_radius = pf.placeholder(name='_radius')
            self._pf_length = pf.placeholder(name='_length')
            self._pf_density = pf.placeholder(name='_density')
            # Put all calculations here
            self._pf_volume = self._pf_length*pi*self._pf_radius**2
            self._pf_mass = self._pf_volume*self._pf_density
            self.graph = graph

    def __setattr__(self, name, value):
        if name in self._dependent:
            raise AttributeError("Cannot set this value.")
        if name in self._independent:
            name = f"_{name}"
        super().__setattr__(name, value)

    def _pf_values(self):
        return {
            self._pf_radius: self._radius,
            self._pf_length: self._length,
            self._pf_density: self._density
        }
    
    @property
    def volume(self):
        """Calculates cylinder's volume."""
        print("Volume calculated")
        return self.graph(self._pf_volume, self._pf_values())

    @property
    def mass(self):
        """Calculates cylinder's mass."""
        print("Mass calculated")
        return self.graph(self._pf_mass, self._pf_values())

    @property
    def length(self):
        return self._length

    @property
    def radius(self):
        return self._radius

    @property
    def density(self):
        return self._density

In [20]:
c = Cylinder(0.25, 1.0, 450)

In [21]:
c.radius

0.25

In [22]:
c.length

1.0

In [23]:
c.density

450

In [24]:
c.volume

Volume calculated


0.19634954084936207

In [25]:
c.mass

Mass calculated


88.35729338221293

In [26]:
c.length = c.length*2  # This should change things!
c.length

2.0

In [27]:
c.mass

Mass calculated


176.71458676442586

In [28]:
c.volume

Volume calculated


0.39269908169872414

In [29]:
try:
    c.volume = 0
except AttributeError as err:
    print(err)

Cannot set this value.


In [30]:
#TODO: Can it cache values?  I don't think so.